# Capítulo 08 — Explorações Extras e Reflexões

Este notebook acompanha o Capítulo 08 da série **Fazendo um LLM do Zero**.
Ele é um laboratório aberto para experimentar limites, comparar estratégias e refletir sobre o comportamento do modelo.

🎯 **Objetivos deste notebook:**
- Limitações dos modelos didáticos
- Impacto do tamanho do dataset
- Comparações entre estratégias
- Geração multi-turno
- Experimentos livres

**Nota importante:** este modelo usa um vocabulário e dataset pequenos.
Entradas fora do vocabulário são normalizadas (acentos removidos e caracteres desconhecidos viram espaço).
Isso pode simplificar a saída — esse efeito é esperado e faz parte do experimento.

![Teoria vs Prática](./infograficos/02-teoria-vs-pratica.png)

---

![Caminhos Futuros](./infograficos/01-caminhos-futuros.png)

---


## 1. Setup e Configuração

In [ ]:
# ============================================================
# Setup do repositório
# ============================================================
import os

REPO_URL = "https://github.com/vongrossi/fazendo-um-llm-do-zero.git"
REPO_DIR = "fazendo-um-llm-do-zero"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

os.chdir(REPO_DIR)
print("Diretório atual:", os.getcwd())


### 1.1 Dependências e Imports

In [ ]:
!pip -q install -r 08-extras/requirements.txt

import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import sys

# Adiciona raiz ao path
sys.path.append(os.getcwd())

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Carregar Modelo Instruction Tuned

In [ ]:
from lib.gptmini import GPTConfig, GPTMini

# ============================================================
# Carregando o modelo treinado (base para os experimentos)
# ============================================================
ckpt_path = None
for p in ["07_instruction_gpt.pt", "gpt_checkpoint.pt",
          "../07-instruction-tuning/07_instruction_gpt.pt", "../gpt_checkpoint.pt"]:
    if os.path.exists(p):
        ckpt_path = p
        break

if ckpt_path is None:
    from google.colab import files
    print("📤 Checkpoint não encontrado. Faça upload do gpt_checkpoint.pt:")
    uploaded = files.upload()
    ckpt_path = next(iter(uploaded.keys()))

ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
stoi, itos = ckpt['stoi'], ckpt['itos']
config = ckpt['config']

model = GPTMini(config).to(device)
model.load_state_dict(ckpt['state_dict'])
print(f"Modelo carregado ✅ | vocab={len(stoi)} | context={config.context_size}")


### 2.1 Funções de Tokenização

In [ ]:
import unicodedata

def normalize_text(text):
    text = text.lower()
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    return text

def sanitize_text(text):
    text = normalize_text(text)
    unk = []
    out = []
    for c in text:
        if c in stoi:
            out.append(c)
        else:
            out.append(' ')
            unk.append(c)
    # colapsa espacos
    s = ' '.join(''.join(out).split())
    return s, sorted(set(unk))

def encode(text):
    text, _ = sanitize_text(text)
    unk_id = stoi.get(' ', 0)
    return [stoi.get(c, unk_id) for c in text]

def decode(tokens):
    return "".join([itos[t] for t in tokens])


**Nota sobre vocabulário e dados**
Este modelo foi treinado com um vocabulário pequeno e dataset didático.
Entradas fora do vocabulário são normalizadas (acentos removidos e caracteres desconhecidos viram espaço).
Por isso, respostas podem parecer simplificadas — isso é esperado e faz parte do experimento.


## 3. Experimento 1 — Impacto do Tamanho da Instrução

Aqui vamos observar como o tamanho do contexto influencia a resposta do modelo.

In [ ]:
@torch.no_grad()
def generate(model, input_text, max_tokens=80, show_sanitized=False):
    safe_text, unk = sanitize_text(input_text)
    if show_sanitized and unk:
        print(f"⚠️ Caracteres fora do vocab: {unk}")
        print(f"➡️ Entrada normalizada: {safe_text}")

    tokens = encode(safe_text)
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    for _ in range(max_tokens):
        idx_cond = tokens[:, -config.context_size:]
        logits, _ = model(idx_cond)
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        tokens = torch.cat([tokens, next_token.unsqueeze(1)], dim=1)
        if itos[next_token.item()] == '\n':
            break

    return decode(tokens.squeeze().tolist())


In [ ]:
input_curto = "explique ia"
input_longo = "explique ia considerando aprendizado supervisionado e nao supervisionado"

print(generate(model, input_curto, show_sanitized=True))
print("\n---\n")
print(generate(model, input_longo, show_sanitized=True))


## 4. Experimento 2 — Limitação de Dataset Pequeno

Vamos observar comportamento fora do domínio treinado.

In [ ]:
print(generate(model, "explique fisica quantica"))


## 5. Experimento 3 — Simulando Conversa Multi-turno

Aqui simulamos uma interação contínua com o modelo.

In [ ]:
conversation = """
usuario: o que e machine learning?
assistente:
"""

print(generate(model, conversation))


## 6. Experimento 4 — Comparar Temperatura

Criatividade vs Determinismo

In [ ]:
def generate_temp(model, input_text, temperature=1.0, max_tokens=80):
    tokens = encode(input_text)
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    for _ in range(max_tokens):
        idx_cond = tokens[:, -config.context_size:]
        logits, _ = model(idx_cond)
        probs = torch.softmax(logits[:, -1, :] / temperature, dim=-1)
        next_token = torch.multinomial(probs, 1)
        tokens = torch.cat([tokens, next_token], dim=1)
        if itos[next_token.item()] == '\n':
            break

    return decode(tokens.squeeze().tolist())


In [ ]:
print(generate_temp(model, "explique ia", temperature=0.3))
print("\n---\n")
print(generate_temp(model, "explique ia", temperature=1.2))


## 7. Experimento 5 — Demonstrando Hallucination

In [ ]:
print(generate(model, "quem descobriu a linguagem python em 1800"))


### 7.1 O Que Esses Experimentos Mostram

Modelos pequenos:

• dependem fortemente do dataset  
• podem produzir respostas plausíveis mas incorretas  
• possuem limitação de contexto  
• demonstram comportamento probabilístico  

Essas características também existem em modelos maiores, embora com menor frequência.


🧪 Experimento 6 — Engenharia de Instrução

In [ ]:
print(generate(model, "responda como professor explicando ia para iniciantes"))


## 9. Visualizando Distribuição de Probabilidades

In [ ]:
show_next_token_distribution(model, "explique ia")


In [ ]:
show_next_token_distribution(model, "Explique IA:")


## 10. Reflexões Finais

Este notebook demonstrou que:

• Modelos didáticos possuem limitações claras  
• Compreender fundamentos permite interpretar comportamento  
• Pequenas mudanças em dados e instruções impactam resultados  
• LLMs são sistemas probabilísticos, não sistemas conscientes  

Este laboratório marca o encerramento da jornada prática da série.
